<a href="https://colab.research.google.com/github/google/business_intelligence_group/blob/development/solutions/x_media_review/X_Media_Review_with_Same_Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# X-Media Review Colab(予実管理)

In [ ]:
sheet_url = "" #@param {type:"string"}
sheet_name = ''  # @param {type:"string"}
# @markdown * * *
fy_start_month = '2024-04'  # @param {type:"string"}

# kpi_target = "" # @param {type:"string"}
kpi_target = "" # @param {type:"string"}
kpi_target = [] if kpi_target == "" else [int(x) for x in kpi_target.split(',')]

# @markdown * * *
market_medium = 'direct', 'referral', 'organic search' # @param
market_medium = list(market_medium)

import datetime
from textwrap import wrap

import pandas as pd
import altair as alt
from altair import datum

import ipywidgets
from IPython.display import display

from google.auth import default
from oauth2client.client import GoogleCredentials
import gspread
from google.colab import auth, files, output

pd.set_option('display.max_columns', 100)

class DataProcessor(object):
    """Base class for cross media data processing."""
    base_metrics = ['Sessions', 'CV', 'Value', 'Investment']
    base_dimension = ['fiscal_year', 'YearMonth', 'Year', 'Month']

    def __init__(self, sheet_url, sheet_name, fy_start_month, kpi_target):
        self.df = self._load_data_from_sheet(sheet_url, sheet_name)
        self._add_dimension_columns(fy_start_month)
        self.monthly_df = self._generate_monthly_df(
            self.df, self.base_dimension, self.base_metrics)
        self._add_kpi_columns_to_monthly_df(kpi_target, fy_start_month)
        self.monthly_by_media_df = self._generate_monthly_df(
            self.df, self.base_dimension + ['Medium'], self.base_metrics)
        self.monthly_by_campaign_df = self._generate_monthly_df(
            self.df,
            self.base_dimension + ['Source', 'Medium', 'Campaign', 'comb'],
            self.base_metrics)
        self.df = self._calculate_composite_metrics(self.df)
        self.month_list = self._generate_fiscal_year_months(fy_start_month)
        self.yearmonth_list = sorted(self.df['YearMonth'].unique().tolist())

        print('media_data.df')
        display(self.df.head())
        print('\n media_data.monthly_df')
        display(self.monthly_df.head())
        print('\n media_data.monthly_by_media_df')
        display(self.monthly_by_media_df.head())
        print('\n media_data.monthly_by_campaign_df')
        display(self.monthly_by_campaign_df.head())
        print('\n media_data.month_list')
        display(self.month_list)
        print('\n media_data.yearmonth_list')
        display(str(self.yearmonth_list))

    @staticmethod
    def _load_data_from_sheet(sheet_url, sheet_name):
        """Load data from Google Spreadsheet.

        Args:
            sheet_url (str): URL of the Google Spreadsheet.
            sheet_name (str): Name of the sheet in the Google Spreadsheet.
            fy_start_month (str): Start month of the fiscal year (YYYY-MM).

        Returns:
            pandas.DataFrame: Loaded dataframe.
        """
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        workbook = gc.open_by_url(sheet_url)
        worksheet = workbook.worksheet(sheet_name)
        df = pd.DataFrame(worksheet.get_all_values())
        df.columns = list(df.loc[0, :])
        df.drop(0, inplace=True)
        df.replace(',', '', regex=True, inplace=True)

        return df

    def _add_dimension_columns(self, fy_start_month):
        self.df['comb'] = (
            self.df.Source + '/' + self.df.Medium + '/' + self.df.Campaign)
        self.df.Month = self.df.Month.apply(lambda x: str(x).zfill(2))
        self.df['YearMonth'] = self.df['Year'].str.cat(
            self.df['Month'], sep='-')
        self.df['fiscal_year'] = (pd.to_datetime(
            self.df['YearMonth'], format='%Y-%m') >= fy_start_month).astype(int)
        self.df[self.base_metrics] = self.df[self.base_metrics].apply(
            pd.to_numeric, downcast='float')
        self.df.fillna(0, inplace=True)

    @staticmethod
    def _calculate_composite_metrics(df):
        """Calculate composite metrics from base metrics.

        This method calculates CPC, CVR, CPA, ROAS, Marginal_Profit,
        ValuePerCV and Marginal_Profit_rate from the base metrics
        (Sessions, CV, Value, Investment).

        Args:
            df (pandas.DataFrame): DataFrame containing the base metrics.

        Returns:
            pandas.DataFrame: DataFrame with the composite metrics.
        """

        composite_metrics = {
            'CPC': df['Investment'] / df['Sessions'],
            'CVR': df['CV'] / df['Sessions'],
            'CPA': df['Investment'] / df['CV'],
            'ROAS': df['Value'] / df['Investment'],
            'Marginal_Profit': df['Value'] - df['Investment'],
            'ValuePerCV': df['Value'] / df['CV'],
            'Marginal_Profit_rate': (
                df['Value'] - df['Investment']) / df['Value'],
            'mROI': (
                df['Value'] - df['Investment']) / df['Investment']
        }
        df_result = pd.concat([df, pd.DataFrame(composite_metrics)], axis=1)
        df_result.fillna(0, inplace=True)

        return df_result

    @staticmethod
    def _generate_monthly_df(df, dimension, metrics):
        grouped_df = df[dimension + metrics].groupby(
            dimension).sum().reset_index()
        grouped_df = DataProcessor._calculate_composite_metrics(grouped_df)

        if dimension != DataProcessor.base_dimension:
            return grouped_df
        else:
            paid_df = df[df['Investment'] > 0][dimension + metrics].groupby(
                dimension).sum().reset_index()
            paid_df = DataProcessor._calculate_composite_metrics(paid_df)

            grouped_df = grouped_df.merge(
                paid_df,
                on=dimension,
                how='outer',
                suffixes=['', '_paid']).assign(
                    **{
                        'Sessions_paid%': lambda x: x[
                            'Sessions_paid'] / x['Sessions'],
                        'CV_paid%': lambda x: x['CV_paid'] / x['CV'],
                        'Value_paid%': lambda x: x['Value_paid'] / x['Value']})
            grouped_df.fillna(0, inplace=True)

        return grouped_df

    def _add_kpi_columns_to_monthly_df(self, kpi_target, fy_start_month):
        """Adds KPI target, rate, and delta columns to the monthly DataFrame.

        Args:
            kpi_target (list): List of KPI target values for each month.
            fy_start_month (str): Start month of the fiscal year (YYYY-MM).
        """
        if not kpi_target:
            self.monthly_df = self.monthly_df.assign(
                kpi_target=0, kpi_rate=0, kpi_delta=0)

            return

        kpi_target_df = pd.DataFrame(
            {
                'YearMonth': pd.date_range(pd.Timestamp(fy_start_month),
                                           periods=12, freq='MS'
                                           ).strftime('%Y-%m'),
                'kpi_target': kpi_target,
                'fiscal_year': 1,}
            ).assign(
                Year=lambda x: x['YearMonth'].str.split('-', expand=True)[0],
                Month=lambda x: x['YearMonth'].str.split('-', expand=True)[1],)

        self.monthly_df = self.monthly_df.merge(
            kpi_target_df,
            on=['YearMonth', 'fiscal_year', 'Year', 'Month'],
            how='outer'
            ).sort_values('YearMonth'
            ).assign(
                kpi_rate=lambda x: x['CV'] / x['kpi_target'],
                kpi_delta=lambda x: x['CV'] - x['kpi_target'],)

    @staticmethod
    def _generate_fiscal_year_months(fy_start_month):
        """Return a list of months in the fiscal year based on the specified date.

        Args:
            fy_start_month (str): Fiscal year start date (YYYY-MM format).

        Returns:
            list: A list of integers representing the months in the fiscal year.
        """

        start_date = datetime.datetime.strptime(fy_start_month, "%Y-%m")
        start_month = start_date.month
        fiscal_year_months = list(range(start_month, 13))
        fiscal_year_months += list(range(1, start_month))

        return fiscal_year_months

class MediaReview(DataProcessor):
    def __init__(self, sheet_url, sheet_name, fy_start_month, kpi_target):
        self.sheet_url = sheet_url
        self.sheet_name = sheet_name
        self.fy_start_month = fy_start_month
        self.kpi_target = kpi_target

        super().__init__(self.sheet_url, self.sheet_name, self.fy_start_month,
                         self.kpi_target)

  # ---- Utility method ----
    @staticmethod
    def _generate_three_axis_chart(
            dataframe,
            month_list,
            metrics_1: str,
            metrics_2: str,
            metrics_3: str,
            height: int,
            width: int):
        """Generate a trend chart for three metrics."""
        AXIS_ONE_RATIO = 2
        chart = []
        # TODO(rhirota) - Set axis limit on CPA case
        if metrics_2 == 'Value':
            axis_one_max = max(
                max(dataframe[metrics_1]), max(dataframe[metrics_2]))
            axis_two_max = axis_one_max

        # TODO(rhirota) - Set axis limit on CPA case
        else:
            axis_one_max = max(dataframe[metrics_1]) * AXIS_ONE_RATIO
            axis_two_max = max(dataframe[metrics_2])

        color_map = ["#DB4437", "#4285F4", "#0F9D58"]

        base_chart = alt.Chart(dataframe).encode(
            x=alt.X(
                'month(Month):O',
                scale=alt.Scale(domain=month_list),
                axis=alt.Axis(format='%b'),
                title=''))

        bars = [
            base_chart.mark_bar(
                size=width / (15 if i == 0 else 25),
                opacity=0.2 if i == 0 else 0.7
                ).encode(
                    alt.Y(
                        metrics_1,
                        axis=alt.Axis(
                            labelColor=color_map[0], titleColor=color_map[0],
                            titleAngle=0, titleAlign="left", titleY=-10),
                        scale=alt.Scale(domain=[0, axis_one_max])),
                    color=alt.ColorValue(color_map[0]),
                ).transform_filter(f"datum.fiscal_year == {i}")
            for i in range(2)]
        chart.append(bars[0] + bars[1])

        lines = [
            base_chart.mark_line(
                opacity=0.6,
                size=1 if i == 0 else 3,
                point={"filled": False, "fill": "white"} if i == 0 else {
                    "filled": False, "fill": color_map[1]},
                strokeDash=[5, 5] if i == 0 else [0]
                ).encode(
                    alt.Y(
                        metrics_2,
                        axis=alt.Axis(
                            labelColor=color_map[1], labelAlign="right",
                            labelPadding=120 if metrics_2 == 'Value' else 75,
                            titleColor=color_map[1], titleAngle=0, tickCount=8,
                            titleAlign="right", titleX=125, titleY=-10,
                            offset=90),
                        scale=alt.Scale(domain=[0, axis_two_max])),
                    color=alt.ColorValue(color_map[1]),
                ).transform_filter(f"datum.fiscal_year == {i}")
            for i in range(2)]
        chart.append(lines[0] + lines[1])

        lines = [
            base_chart.mark_line(
                opacity=0.6,
                size=1 if i == 0 else 2,
                strokeDash=[5, 5] if i == 0 else [0]
                ).encode(
                    alt.Y(
                        metrics_3,
                        axis=alt.Axis(
                            labelColor=color_map[2], labelAlign="right",
                            labelPadding=65, titleAngle=0, titleY=-10,
                            titleColor=color_map[2], titleAlign="right",
                            format="%" if metrics_3 in [
                                "ROAS", "Marginal_Profit_rate"] else ",.0f",)),
                    color=alt.ColorValue(color_map[2]),
                ).transform_filter(f"datum.fiscal_year == {i}")
            for i in range(2)]
        chart.append(lines[0] + lines[1])

        return chart

    @staticmethod
    def _generate_YearMonth_pre_post_df(df, to_index, pre, post):
        pre_post_df = df.copy()
        mapping = {pre: 'pre', post: 'post'}
        pre_post_df.loc[:, 'YearMonth'] = pre_post_df[
            'YearMonth'].replace(mapping)

        pre_post_df = pre_post_df.query(
            'YearMonth in @mapping.values()').pivot_table(
            index=to_index, columns='YearMonth', aggfunc='sum', fill_value=0)
        pre_post_df.columns = [
            '_'.join(y).replace(',', '_') for y in pre_post_df.columns.values]
        pre_post_df.reset_index(inplace=True)

        return pre_post_df

  # ---- visualize_monthly_df & visualize_monthly_trend_chart ----
    @staticmethod
    def visualize_monthly_df(monthly_df):
        """Visualize the monthly DataFrame.

        Args:
            monthly_df (pandas.DataFrame): DataFrame to be visualized.
        """
        display_columns = ['YearMonth', 'Investment', 'CV', 'CV_paid%', 'CPA',
            'CPA_paid', 'Value', 'Value_paid%', 'ROAS', 'ROAS_paid',
            'Marginal_Profit', 'Marginal_Profit_rate', 'ValuePerCV', 'CPC',
            'CVR']

        if not kpi_target:
            pass
        else:
            display_columns[1:1] = ['kpi_target', 'kpi_rate', 'kpi_delta']
            display(alt.Chart(
                monthly_df.query('fiscal_year == 1 & Sessions > 0')
                ).transform_fold(
                    ['CV', 'kpi_target', 'kpi_delta']
                ).mark_bar(size=30).encode(
                    y=alt.Y(
                        'key:N', title='',
                        sort=['kpi_target', 'CV', 'kpi_delta']
                        ),
                    x=alt.X('sum(value):Q', title=''),
                    color=alt.Color(
                        'key:N',
                        scale=alt.Scale(
                            range=['#4285F4', '#54A6CF', '#BDE7F0'])),
                ).properties(height=150, width=400))

        display(monthly_df[display_columns].style.format({
            'kpi_target': '{:,.0f}',
            'kpi_rate': '{:,.2%}',
            'kpi_delta': '{:,.0f}',
            'Investment': '{:,.0f}',
            'CV': '{:,.0f}',
            'CV_paid%': '{:,.2%}',
            'CPA': '{:,.0f}',
            'CPA_paid': '{:,.0f}',
            'Value': '{:,.0f}',
            'Value_paid%': '{:,.2%}',
            'ROAS': '{:,.2%}',
            'ROAS_paid': '{:,.2%}',
            'Marginal_Profit': '{:-,.0f}',
            'Marginal_Profit_rate': '{:-,.2%}',
            'ValuePerCV': '{:-,.0f}',
            'CPC': '{:,.0f}',
            'CVR': '{:,.2%}',
            }, na_rep="-"))

    @staticmethod
    def visualize_monthly_trend_chart(monthly_df, month_list, kpi_target):
        HEIGHT = 250
        WIDTH = 350
        LABEL_SIZE = 12
        TITLE_SIZE = 15

        # TODO(rhirota)-Check Metrics matching
        metrics_data = {
            "CV": {"metrics_1": "Investment", "metrics_2": "CPA"},
            "Value": {"metrics_1": "Investment", "metrics_2": "ROAS"},
            "ValuePerCV": {"metrics_1": "Marginal_Profit",
                           "metrics_2": "Marginal_Profit_rate"},}

        for metrics_2, metrics in metrics_data.items():
            chart = media_data._generate_three_axis_chart(
                monthly_df, month_list,
                metrics["metrics_1"], metrics_2, metrics["metrics_2"],
                height=HEIGHT, width=WIDTH)

            if kpi_target and metrics_2 == "CV":
                kpi_trend = alt.Chart(monthly_df).mark_tick(
                    color='#CCCCCC', thickness=3, size=15,
                    ).encode(
                        x=alt.X(
                            'month(Month):O',
                            scale=alt.Scale(domain=month_list),
                            axis=alt.Axis(format='%b'), title=''),
                        y=alt.Y('kpi_target', title='CV'),
                    ).properties(height=HEIGHT, width=WIDTH)
                chart[1] = chart[1] + kpi_trend

            print('\n')
            display(alt.layer(
                chart[0], chart[1], chart[2]
                ).resolve_scale(y='independent'
                ).properties(height=HEIGHT, width=WIDTH,
                ).configure_axis(
                    labelFontSize=LABEL_SIZE, titleFontSize=TITLE_SIZE,
                    grid=False))

  # ---- visualize_demand_chart ----
    @staticmethod
    def visualize_demand_chart(monthly_by_media_df):
        base_chart = alt.Chart(monthly_by_media_df).encode(
            x=alt.X(
                'month(Month):O', scale=alt.Scale(domain=media_data.month_list),
                axis=alt.Axis(format='%b'), title=''))

        chart = []
        areas = [
            base_chart.mark_area(
                line=True if i == 0 else False,
                opacity=0.1 if i == 0 else 0.8
                ).encode(
                    alt.Y(
                        'Sessions:Q',
                        axis=alt.Axis(
                            labelColor='#4285F4', titleColor='#4285F4',
                            titleAngle=0, titleAlign='left', titleY=-10,),),
                    color=alt.Color(
                        'Medium', sort=market_medium,
                        scale=alt.Scale(
                            range=['#4285F4', '#54A6CF', '#BDE7F0']),
                        legend=alt.Legend(
                            orient='none', title='', legendX=-40, legendY=-40,
                            direction='horizontal', titleAnchor='start'),),
                    order=alt.Order('color_Medium_sort_index:Q')
                ).transform_filter(f"datum.fiscal_year == {i}")
            for i in range(2)]
        chart.append(areas[0] + areas[1])

        lines = [
            base_chart.mark_line(
                point=False if i == 0 else True,
                opacity=0.5 if i == 0 else 1,
                strokeDash=[5, 5] if i == 0 else [0],
                ).encode(
                    alt.Y(
                        'cvr:Q',
                        axis=alt.Axis(
                            format='%', labelColor='#DB4437',
                            titleColor='#DB4437', titleAngle=0,
                            titleAlign='right', titleY=-10,),),
                    color=alt.ColorValue('#DB4437'),
                ).transform_aggregate(
                    session='sum(Sessions)',
                    cv='sum(CV)',
                    groupby=["Month", "fiscal_year"]
                ).transform_calculate(cvr='datum.cv/datum.session'
                ).transform_filter(f'datum.fiscal_year == {i}')
            for i in range(2)]
        chart.append(lines[0] + lines[1])

        display(alt.layer(chart[0], chart[1]).resolve_scale(
            y='independent').properties(width=600, height=300))

  # ---- visualize_mom_stats ----
    @staticmethod
    def _generate_water_fall_chart(df, value, pre, post):
        """Generate a waterfall chart showing the change in a value."""
        delta_col = f"{value}_delta"
        wtf_df = df[['label', delta_col]]
        wtf_df.columns = ['label', 'amount']

        # データの準備
        start_data = pd.DataFrame(
            {'label': [pre], 'amount': [df[f"{value}_pre"].sum()]})
        end_data = pd.DataFrame({'label': [post], 'amount': [0]})
        wtf_df = pd.concat([start_data, wtf_df, end_data], ignore_index=True)
        wtf_df['amount'] = wtf_df['amount'].astype('float')

        if value == 'Value':
            wtf_df['amount'] = wtf_df['amount'] / 1000000

        # チャートの作成
        base_chart = alt.Chart(wtf_df).transform_window(
            window_sum_amount="sum(amount)",
            window_lead_label="lead(label)",
            ).transform_calculate(
                prev_sum=f"datum.label === '{post}' ? 0 : datum.window_sum_amount - datum.amount",
                amount=f"datum.label === '{post}' ? datum.window_sum_amount : datum.amount",
                text_amount="format((datum.label !== '{pre}' && datum.label !== '{post}') + datum.amount, ',.0f')",
                center="(datum.window_sum_amount + datum.prev_sum) / 2",
            ).encode(
                x=alt.X("label:O", axis=alt.Axis(
                    labelAngle=30, labelFontSize=15), sort=None, title=""))

        color_coding = {
            "condition": [
                {"test": f"datum.label === '{pre}' || datum.label === '{post}'",
                 "value": "#878d96"},
                {"test": "datum.amount < 0", "value": "#24a148"},],
            "value": "#fa4d56",}

        bar = base_chart.mark_bar(size=45).encode(
            y=alt.Y("prev_sum:Q", title="", axis=alt.Axis(labelFontSize=15)),
            y2=alt.Y2("window_sum_amount:Q"),
            color=color_coding,)
        text = base_chart.mark_text(baseline="top", dy=10, size=15,).encode(
            text=alt.Text("text_amount:N"), y=alt.Y("center:Q"))
        wtf_chart = alt.layer(bar, text
                              ).configure_axis(grid=False
                              ).configure_view(strokeWidth=0
                              ).properties(width=700, height=100)

        return wtf_chart

    @staticmethod
    def _generate_top_n_df(mom_df, kpi, topN=9):
        change_metrics = ['Sessions', 'CV', 'Value', 'Investment', 'CPC', 'CVR',
                          'CPA', 'ROAS', 'Marginal_Profit', 'ValuePerCV']

        _df_largest = mom_df.nlargest(topN, columns=f"{kpi}_post")
        _df_others = pd.DataFrame(
            mom_df.drop(_df_largest.index).sum(numeric_only=True)).T
        topN_df = pd.concat(
            [_df_largest, _df_others], ignore_index=True
            ).fillna('Others').sort_values(f"{kpi}_post", ascending=False)

        for metric in change_metrics:
            topN_df[f"{metric}_change"] = (
                topN_df[f"{metric}_post"] / topN_df[f"{metric}_pre"] - 1)

        topN_df[f"{kpi}_delta"] = topN_df[f"{kpi}_post"] - topN_df[f"{kpi}_pre"]

        return topN_df

    @staticmethod
    def _generate_stats_change(mom_df, kpi):
        axis_labels = (
            """datum.label == 'CV_change' ? 'CV'
            : datum.label == 'CVR_change' ? 'CVR'
            : datum.label == 'Sessions_change' ? 'Sessions'
            : datum.label == 'Investment_change' ? 'Investment'
            : 'Other'
            """)
        delta_col = f"{kpi}_delta"
        metrics_col = [
            'CV_change', 'CVR_change', 'Sessions_change', 'Investment_change']
        order = mom_df.sort_values(
            delta_col, ascending=False)['Medium'].to_list()
        base = alt.Chart(mom_df).transform_fold(metrics_col).encode(
            x=alt.X(
                'key:N',
                axis=alt.Axis(
                    labelAngle=15, labelFontSize=12,labelExpr=axis_labels,
                    # labelPadding=-10,
                    labelOffset=-15
                    ),
                sort=metrics_col, title=''),
            y=alt.Y('value:Q', title='', axis=None, scale=alt.Scale(
                        domain=[-0.5, 0.5], clamp=True),))
        points = base.mark_circle(size=100).encode(
            color=alt.Color('key:N', legend=None)
            ).properties(width=200, height=40)
        text = base.mark_text(
            align='center', baseline='middle', dy=-15, size=15,
            ).encode(text=alt.Text('value:Q', format='.0%'), color='key:N')
        rule = alt.Chart(
            pd.DataFrame({'value': ['0'], 'color': ['red']})
            ).mark_rule(strokeDash=[5, 5]).encode(
                y='value:Q', color=alt.Color('color:N', scale=None))

        chart = (points + text + rule).facet(
            facet=alt.Facet('Medium', sort=order, header=alt.Header(title='',
                labelAlign='center', labelAnchor='start', labelFontSize=15,)),
            spacing=10, columns=3).configure_view(stroke=None)

        return chart

    @staticmethod
    def visualize_mom_stats(monthly_by_media_df, yearmonths, kpi):
        if len(yearmonths) == 1:
            print(yearmonths)
            print('複数時点のデータポイントが必要です。')

        else:
            sub_tab = [ipywidgets.Output() for i in yearmonths[1:]]
            tab_option = ipywidgets.Tab(sub_tab)

            for i in range(len(yearmonths) - 1):
                tab_option.set_title(i, yearmonths[i] + ' & ' + yearmonths[i+1])
                mom_media_df = media_data._generate_YearMonth_pre_post_df(
                    monthly_by_media_df, ['Medium'],
                    yearmonths[i], yearmonths[i+1])

                topN_df = media_data._generate_top_n_df(
                    mom_media_df, kpi, topN=9)

                medium_water_fall_chart = media_data._generate_water_fall_chart(
                    topN_df.rename(columns={'Medium': 'label'}
                        ).sort_values(f"{kpi}_delta", ascending=False),
                    kpi, yearmonths[i], yearmonths[i+1])

                medium_change_chart = media_data._generate_stats_change(
                    topN_df[topN_df['Medium'] != 'Others'], kpi)

                with sub_tab[i]:
                    display(medium_water_fall_chart)
                    display(medium_change_chart)

                tab_option.selected_index = i
            display(tab_option)

  # ---- visualize_mom_roi ----
    @staticmethod
    def _generate_mom_table(mom_df: pd.DataFrame) -> dict:

        def calculate_average(df, numerator, denominator):
            """Calculates and returns average metrics."""
            avg_pre = (
                sum(df[f"{numerator}_pre"]) / sum(df[f"{denominator}_pre"])
                if sum(df[f"{denominator}_pre"]) else 0)
            avg_post = (
                sum(df[f"{numerator}_post"]) / sum(df[f"{denominator}_post"])
                if sum(df[f"{denominator}_post"]) else 0)

            return avg_pre, avg_post

        def print_mom_change(metric_name, pre, post):
            """Prints the MoM change for a given metric."""
            change = post - pre
            percentage_change = (post / pre - 1) if pre else 0
            format = ',.0f' if metric_name != 'ROAS' else ',.2%'
            print(
                f'{metric_name:<15}: {pre:>15{format}} >> {post:>15{format}}'
                f'({change:+15{format}}|{percentage_change:+8,.2%})')

        aggregate_metrics = {
            "Investment": "Investment",
            "Value": "Value",
            "Conversion": "CV"}

        composit_metrics = {
            "CPA": ("Investment", "CV"),
            "ROAS": ("Value", "Investment"),
            "Value/CV": ("Value", "CV")}

        roi_dict = {}

        for name, metrics in aggregate_metrics.items():
            pre = mom_df[f"{metrics}_pre"].sum()
            post = mom_df[f"{metrics}_post"].sum()
            print_mom_change(name, pre, post)

        for name, (numerator, denominator) in composit_metrics.items():
            pre, post = calculate_average(mom_df, numerator, denominator)
            print_mom_change(name, pre, post)
            roi_dict[f'average_{name.lower()}_pre'] = pre
            roi_dict[f'average_{name.lower()}_post'] = post

        return roi_dict

    @staticmethod
    def _generate_scatter_plot(mom_df, roi_dict, x_axis_max, y_axis_max, j):

        def create_rule_chart(name, axis, value, color='red'):
            df = pd.DataFrame({name: [str(value)], 'color': [color]})
            return (
                alt.Chart(df).mark_rule(strokeDash=[5, 5]).encode(
                    **{axis: f'{name}:Q'},
                    color=alt.Color('color:N', scale=None)))

        def create_bubble_chart(mom_df, x_axis_max, y_axis_max, j, source=None):
            """Creates a bubble chart with tooltips."""

            base_chart = alt.Chart(mom_df.query(f'CPA{j} > 0')).encode(
                    x=alt.X(
                        f'ROAS{j}', axis=alt.Axis(format='%'), title='ROAS',
                        scale=alt.Scale(domain=[0, x_axis_max], clamp=True),),
                    y=alt.Y(
                        f'CPA{j}', title='CPA',
                        scale=alt.Scale(domain=[0, y_axis_max], clamp=True),),
                    color=alt.Color(
                        'Medium',
                        legend=alt.Legend(
                            orient='none', title='', legendX=-40, legendY=-40,
                            direction='horizontal', titleAnchor='start')),
                    size=alt.Size(
                        f'Investment{j}', legend=None,
                        scale=alt.Scale(range=[100, 1000]),),
                    tooltip=[
                        alt.Tooltip(field='comb'),
                        alt.Tooltip(field=f'Investment{j}', format=',.0f'),
                        alt.Tooltip(field=f'CV{j}', format=',.2f'),
                        alt.Tooltip(field=f'CPA{j}', format=',.0f'),
                        alt.Tooltip(field=f'Value{j}', format=',.0f'),
                        alt.Tooltip(field=f'ROAS{j}', format=',.0%'),
                        alt.Tooltip(field=f'ValuePerCV{j}', format=',.0f'),],
                ).properties(width=400)

            if source:
                return base_chart.transform_filter(
                    alt.FieldEqualPredicate(field='Source', equal=source)
                    ).mark_point(filled=True)
            else:
                return base_chart.mark_point()

        chart = []
        average_roas = create_rule_chart(
            'roas', 'x', roi_dict[f"average_roas{j}"], )
        average_cpa = create_rule_chart(
            'cpa', 'y', roi_dict[f"average_cpa{j}"])
        bubble = create_bubble_chart(mom_df, x_axis_max, y_axis_max, j)
        fill = create_bubble_chart(
            mom_df, x_axis_max, y_axis_max, j, source="google")

        chart.append(average_roas)
        chart.append(average_cpa)
        chart.append(bubble)
        chart.append(fill)

        return chart

    @staticmethod
    def _list_top_and_bottom(mom_df):
        text_configs = {
            'Source': {'width': 80},
            'Medium': {'width': 50},
            'Campaign': {'width': 50},}

        cpa_configs = {
            'Investment_post': {
                'color': "#DB4437", 'title': 'Investment', 'format': ',.0f'},
            'CV_post': {'color': "#4285F4", 'title': 'CV', 'format': ',.1f'},
            'CPA_post': {'color': "#0F9D58", 'title': 'CPA', 'format': ',.0f'},
            'CPA_pre': {
                'color': "#0F9D58", 'title': '前期CPA', 'format': ',.0f'},}

        roas_configs = {
            'Investment_post': {
                'color': "#DB4437", 'title': 'Investment', 'format': ',.0f'},
            'Value_post': {
                'color': "#4285F4", 'title': 'Value', 'format': ',.0f'},
            'ROAS_post': {
                'color': "#0F9D58", 'title': 'ROAS', 'format': ',.1%'},
            'ROAS_pre': {
                'color': "#0F9D58", 'title': '前期ROAS', 'format': ',.1%'},}

        for i in ['CPA', 'ROAS']:
            ordered_df = mom_df[mom_df['Investment_post']>0].sort_values(
                f'{i}_post', ascending=False
                ).reset_index(drop=True).reset_index()

            for j in range(2):
                order = 'ascending' if j == 0 else 'descending'
                pick = ordered_df.head(10) if j == 0 else ordered_df.tail(10)

                charts = []
                chart_dict = cpa_configs if i == 'CPA' else roas_configs
                for key, config in text_configs.items():
                    chart = alt.Chart(pick).mark_text(
                        size=15, align='left', limit=150).encode(
                            y=alt.Y(
                                'comb:N', title=f'{key}',
                                axis=alt.Axis(
                                    labels=False, domain=False, ticks=False,
                                    titleAngle=0, titleAlign='left', titleX=30,
                                    titleY=-10),
                                sort=alt.EncodingSortField(
                                    field="index", order=order)),
                            text=alt.Text(f'{key}:N')
                        ).properties(height=400, width=config['width'])
                    charts.append(chart)

                for key, config in chart_dict.items():
                    chart = alt.Chart(pick).mark_bar(
                        size = 15, opacity=0.6).encode(
                            y=alt.Y(
                                'comb:N', title=config['title'],
                                axis=alt.Axis(
                                    labels=False, titleAngle=0, titleAlign='left',
                                    titleX=0, titleY=-10),
                                sort=alt.EncodingSortField(
                                    field="index", order=order)),
                            x=alt.X(f'{key}', title='', axis=None,),
                            color=alt.ColorValue(config['color'])
                        ).properties(height=400, width=50)
                    text = chart.mark_text(align='left', dx=3, size = 14).encode(
                        text=alt.Text(f'{key}', format=config['format']))
                    charts.append(chart+text)

                print('\n')
                display(
                    (charts[0]|charts[1]|charts[2]
                    |charts[3]|charts[4]|charts[5]|charts[6]
                    ).resolve_scale(y='shared'
                    ).configure_view(strokeWidth=0
                    ).configure_axis(grid=False, labelFontSize=13))

            print('-'*120)

    @staticmethod
    def visualize_mom_roi(monthly_df, yearmonths, x_axis_max, y_axis_max):
        if len(yearmonths) == 1:
            print(yearmonths)
            print('複数時点のデータポイントが必要です。')

        else:
            sub_tab = [ipywidgets.Output() for i in yearmonths[1:]]
            tab_option = ipywidgets.Tab(sub_tab)
            for i in range(len(yearmonths) - 1):
                tab_option.set_title(i, yearmonths[i] + ' & ' + yearmonths[i+1])

                with sub_tab[i]:
                    mom_comb_df = media_data._generate_YearMonth_pre_post_df(
                        monthly_df.query('Investment > 0'),
                        ['Source', 'Medium', 'Campaign', 'comb'],
                        yearmonths[i], yearmonths[i+1])

                    roi_dict = media_data._generate_mom_table(mom_comb_df)

                    print('\n')
                    scatter = []
                    for j in ['_pre', '_post']:
                        scatter += media_data._generate_scatter_plot(
                            mom_comb_df, roi_dict, x_axis_max, y_axis_max, j)

                    display(alt.hconcat(
                        (scatter[0] + scatter[1] + scatter[2] + scatter[3]),
                        (scatter[4] + scatter[5] + scatter[6] + scatter[7])
                         ).configure_axis(grid=False, labelFontSize=13,))

                    media_data._list_top_and_bottom(mom_comb_df)

                tab_option.selected_index = i
            display(tab_option)

  # ---- visualize_target_trend ----
    @staticmethod
    def generate_target_trend(monthly_df, campaign_df, target_keys):
        HEIGHT = 250
        WIDTH = 700
        LABEL_SIZE = 15
        TITLE_SIZE = 15
        metrics_columns = [
            'Sessions', 'CPC', 'CVR', 'CPA', 'ValuePerCV', 'ROAS', 'mROI']
        paid_columns = [column + "_paid" for column in metrics_columns]

        monthly_paid_average = monthly_df[
            media_data.base_dimension + paid_columns
            ].query('Sessions_paid > 0')
        recent_month = monthly_paid_average.YearMonth[-3:].tolist()

        for i in target_keys:
            chart = []
            display(i)
            print('\n')
            target_df = campaign_df.query('comb == @i')
            chart = media_data._generate_three_axis_chart(
                target_df, media_data.month_list,
                'Investment', 'CV', 'CPA', HEIGHT, WIDTH)

            display(alt.layer(
                chart[0], chart[1], chart[2]
                ).resolve_scale(y='independent'
                ).properties(height=HEIGHT, width=WIDTH,
                ).configure_axis(
                    labelFontSize=LABEL_SIZE, titleFontSize=TITLE_SIZE,
                    grid=False))

            recent_chart = []
            for j in metrics_columns:
                chart = alt.Chart(
                    target_df.query("YearMonth in @recent_month")
                    ).mark_line(point=True).encode(
                        x=alt.X('month(Month):O', title='', sort=recent_month),
                        y=alt.Y(j, title=j, axis=alt.Axis(
                            titleAngle=0, titleAlign='left',
                            titleX=-10, titleY=-10)),
                        color=alt.value('#4385F4')
                    ).properties(height=130, width=70)

                if j in ['Sessions']:
                    recent_chart.append(chart)
                else:
                    recent = alt.Chart(
                        monthly_paid_average.query("YearMonth in @recent_month")
                        ).mark_line(strokeDash=[5, 5], point=True).encode(
                            x=alt.X('month(Month):O', sort=recent_month),
                            y=alt.Y(f'{j}'+'_paid', title='',),
                            color=alt.value('gray')
                        ).properties(height=130, width=70)
                    recent_chart.append(chart + recent)

            display(
                alt.hconcat(
                    recent_chart[0], recent_chart[1], recent_chart[2],
                    recent_chart[3], recent_chart[4], recent_chart[5],
                    recent_chart[6]))

  # ---- zzz ----
    @staticmethod
    def change_contribution(df, roi_dict):
        colors = {
            ('positive', 'positive'): '#ea4335',
            ('positive', 'negative'): '#4285f4',
            ('negative', 'positive'): '#34a853',
            ('negative', 'negative'): '#fbbc04'}

        print('\n' + '='*100)
        print('KPI Chnage Factor Analysis \n')

        df['CPA_diff'] = df['CPA_post'] / roi_dict["_average_cpa_pre"] - 1
        df['CPA_diff'] = df['CPA_diff'].where(df['CPA_diff'] <= 3, 3)
        df['Invest_share_post'] = (
            df['Investment_post'] / df['Investment_post'].sum())

        df['Invest_share_pre'] = (
            df['Investment_pre'] / df['Investment_pre'].sum())

        df['Invest_share_delta'] = (
            df['Invest_share_post'] - df['Invest_share_pre'])

        df['change_factor_index'] = (
            df['CPA_diff'] * df['Invest_share_delta'])

        df['color'] = df.apply(
            lambda row: colors[
                ('positive' if row[
                    'CPA_diff'] >= 0 else 'negative',
                'positive' if row[
                    'Invest_share_delta'] >= 0 else 'negative')
            ], axis=1)

        df['abs_index'] = df['change_factor_index'].abs()
        df.sort_values('change_factor_index', ascending=False, inplace=True)

        contribution_map = (
            alt.Chart(df).mark_point().encode(
                x=alt.X(
                    'Invest_share_delta',
                    title='Investment Delta',
                    axis=alt.Axis(format='%')),
                y=alt.Y(
                    'CPA_diff',
                    title='CPA Delta',
                    axis=alt.Axis(format='%'),),
                color=alt.Color('color', scale=None),
                size=alt.Size(
                    'abs_index',
                    scale=alt.Scale(range=[10, 500]),
                    legend=None),
                tooltip=[
                    alt.Tooltip(field='key'),
                    alt.Tooltip(field='CPA_diff', format='.1%'),
                    alt.Tooltip(
                        field='Invest_share_delta', format='.1%'),
                    alt.Tooltip(
                        field='change_factor_index', format='.1%')])
            .properties(width=500, height=400)
            + alt.Chart(df.query(' Source == "google" ')).mark_point(
                filled=True).encode(
                    x=alt.X(
                        'Invest_share_delta',
                        title='Investment Delta',
                        axis=alt.Axis(format='%')),
                    y=alt.Y(
                        'CPA_diff',
                        title='CPA Delta',
                        axis=alt.Axis(format='%'),),
                    color=alt.Color('color', scale=None),
                    size=alt.Size(
                        'abs_index',
                        scale=alt.Scale(range=[10, 500]),
                        legend=None),
                    tooltip=[
                        alt.Tooltip(field='key'),
                        alt.Tooltip(field='CPA_diff', format='.1%'),
                        alt.Tooltip(
                            field='Invest_share_delta', format='.1%'),
                        alt.Tooltip(
                            field='change_factor_index', format='.1%')])
            .properties(width=500, height=400)
            + alt.Chart(
                pd.DataFrame({'zero': [str(0)]})).mark_rule().encode(
                    x=alt.X('zero:Q', title=''))
            + alt.Chart(
                pd.DataFrame({'zero': [str(0)]})).mark_rule().encode(
                    y=alt.Y('zero:Q', title='')))

        display(contribution_map.configure_axis(labelFontSize=13))

        display(
            alt.Chart(
                pd.concat([df.head(10), df.tail(10)]))
                .mark_rule(size=5).encode(
                    x=alt.X(
                        'change_factor_index',
                        axis=alt.Axis(format='%')),
                    y=alt.Y('key', sort='-x', title=''),
                    color=alt.Color('color', scale=None),
                    ).properties(width=500, height=400).configure_axis(
                        labelLimit=300,
                        labelFontSize=13,
                    ).configure_axisY(
                        titleAngle=0,
                        titleY=-10,
                        titleX=-60,
                        labelPadding=300,
                        labelAlign='left',))

output.no_vertical_scroll()
media_data = MediaReview(sheet_url, sheet_name, fy_start_month, kpi_target)

In [ ]:
# @markdown Paid Media Stats Table for download
media_data.df[media_data.df['Investment']>0]

In [ ]:
# @markdown visualize_monthly_df & visualize_monthly_trend_chart
output.no_vertical_scroll()
media_data.visualize_monthly_df(media_data.monthly_df)
media_data.visualize_monthly_trend_chart(
    media_data.monthly_df, media_data.month_list, media_data.kpi_target)

In [ ]:
# @markdown monthly_by_media_df
source = media_data.monthly_by_media_df.query("Medium in @market_medium")
media_data.visualize_demand_chart(source)

In [ ]:
# @markdown visualize_mom_stats
output.no_vertical_scroll()
kpi = "CV" # @param ["CV","Value"]

yearmonths = media_data.yearmonth_list[-4:]

# TODO(rhirota) - target_months選択式
# yearmonths = ['2024-04', '2025-01']
# target_months = "'2024-01','2025-01'" # @param {"type":"string"}
# target_months = [date.strip("'") for date in target_months.split(", ")]
# yearmonths = media_data.yearmonth_list if len(target_months) == 1 else target_months
print(yearmonths)

media_data.visualize_mom_stats(media_data.monthly_by_media_df, yearmonths, kpi)

In [ ]:
# @markdown visualize_mom_roi
output.no_vertical_scroll()
x_axis_max = 10 # @param {"type":"integer"}
y_axis_max = 150000 # @param {"type":"integer"}
media_data.visualize_mom_roi(
    media_data.monthly_by_campaign_df, yearmonths, x_axis_max, y_axis_max)

In [ ]:
# @markdown visualize Paid Investment by YearMonth
media_data.monthly_by_campaign_df[
    media_data.monthly_by_campaign_df['Investment']>0
    ].pivot(
        index = ["Medium","Source","Campaign"],
        columns='YearMonth',
        values='Investment')

In [ ]:
# @markdown Select Target Source/Medium/Campaign
key_list = ipywidgets.SelectMultiple(
    options=sorted(
        media_data.df[media_data.df['Investment']>0].comb.unique().tolist()),
    description='Source/Medium/Campaign',
    disabled=False,
    layout=ipywidgets.Layout(height='500px', width='500px'),)

display(key_list)

In [ ]:
# @markdown generate_target_trend
output.no_vertical_scroll()
target_keys = list(key_list.value)
media_data.generate_target_trend(
    media_data.monthly_df, media_data.monthly_by_campaign_df,
    target_keys)